In [10]:
import pandas as pd
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from huggingface_hub import login
import torch
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()
login(token=os.getenv("HUGGINGFACE_TOKEN"))

In [12]:
# Download VADER lexicon
nltk.download("vader_lexicon")

# Load CSV
path = '../data/'
df = pd.read_csv(path + "test.csv")
texts = df["user_review"].astype(str).tolist()
labels = df["user_suggestion"].tolist()

### 1. DISTILBERT Evaluation
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)  # Use GPU if available

predictions = classifier(texts, truncation=True, padding=True)
label_map = {"NEGATIVE": 0, "POSITIVE": 1}
bert_preds = [label_map[p["label"]] for p in predictions]

print("Distilbert")
print(classification_report(labels, bert_preds, digits=4))

### 2. Fine-tuned DistilBERT Evaluation
checkpoint = "GaaS-Team/DistilBERT-finetuned-GaaS"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
ft_classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)  # Use GPU if available

ft_predictions = ft_classifier(texts, truncation=True, padding=True)
ft_bert_preds = [label_map[ftp["label"]] for ftp in ft_predictions]

print("Fine-tuned Distilbert")
print(classification_report(labels, ft_bert_preds, digits=4))

### 3. VADER Evaluation
vader = SentimentIntensityAnalyzer()

def vader_to_label(text):
    score = vader.polarity_scores(text)["compound"]
    return 1 if score >= 0 else 0  # 1 = positive, 0 = negative

vader_preds = [vader_to_label(text) for text in texts]

print("Vader")
print(classification_report(labels, vader_preds, digits=4))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Device set to use cuda:0


Distilbert
              precision    recall  f1-score   support

           0     0.7747    0.7703    0.7725      3348
           1     0.6149    0.6208    0.6179      1978

    accuracy                         0.7148      5326
   macro avg     0.6948    0.6956    0.6952      5326
weighted avg     0.7154    0.7148    0.7151      5326



Device set to use cuda:0


Fine-tuned Distilbert
              precision    recall  f1-score   support

           0     0.8944    0.9259    0.9099      3348
           1     0.8667    0.8150    0.8400      1978

    accuracy                         0.8847      5326
   macro avg     0.8805    0.8704    0.8750      5326
weighted avg     0.8841    0.8847    0.8839      5326

Vader
              precision    recall  f1-score   support

           0     0.9040    0.4050    0.5594      3348
           1     0.4794    0.9272    0.6320      1978

    accuracy                         0.5989      5326
   macro avg     0.6917    0.6661    0.5957      5326
weighted avg     0.7463    0.5989    0.5864      5326

